# [Module 3.1] Personalize 솔류션 및 솔류션 버전 생성

이 노트북은 Module1에서 생성한 데이타셋 그룹, 데이타셋을 바탕으로 아래와 같은 작업을 합니다.

* 레서피(알고리즘) 선택 및 7가지 솔류션 생성
* 4가지 솔류션 버전 생성

--- 
이 노트북은 약 50분 소요 됩니다.



In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time


다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [3]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [4]:
suffix = str(np.random.uniform())[4:9]

## 솔루션 및 버전 생성

Amazon Personalize에서 훈련된 모델을 솔루션이라고 하며, 각 솔루션에는 모델이 훈련되었을 때 주어진 데이터량과 관련된 많은 특정 버전들이 있을 수 있습니다.

우선, Amazon Personalize에서 지원되는 모든 레시피(레시피는 아직 데이터에 대해 훈련되지 않은 알고리즘입니다.)들을 나열합니다. 리스트업된 레시피들 중 하나를 선택하고 이를 사용하여 모델을 빌드해 보세요.

이 LAB에서 아래와 같은 모든 솔루션을 생성하여 성능을 비교하여 봅니다.

    1) User-Personalization 
    2) popularity-count 
    3) similar-items
    4) Personalized Ranking

이 프로세스의 완료는 실제로 40분 이상 소요됩니다. 작업이 완료될 때까지(즉, 활성화 상태가 될 때까지) while 루프를 수행하는 방법도 있지만, 이렇게 하면 다른 셀의 실행을 차단하게 됩니다. 따라서, 많은 모델을 만들어 신속하게 배포하려면 while 루프를 사용하는 대신, 필요한 솔루션 버전들을 생성 후, SageMaker 및 Cloudwatch에서 업데이트를 확인하세요.

###  레시피 리스트 확인

In [6]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed
arn:aws:personalize:::recipe/aws-ecomm-frequently-bought-together
arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-purchases
arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-views
arn:aws:personalize:::recipe/aws-ecomm-recommended-for-you
arn:aws:personalize:::recipe/aws-item-affinity
arn:aws:personalize:::recipe/aws-item-attribute-affinity
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-similar-items
arn:aws:personalize:::recipe/aws-sims


###  User-Personalization 솔루션 생성 

In [7]:
# Recipe 선택 
system_user_personalization_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-user-personalization-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_user_personalization_recipe_arn,
)

In [8]:
user_personalization_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-user-personalization-08866",
  "ResponseMetadata": {
    "RequestId": "86480f70-24d5-4edd-a20e-95dbb9f20073",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:10:14 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "111",
      "connection": "keep-alive",
      "x-amzn-requestid": "86480f70-24d5-4edd-a20e-95dbb9f20073"
    },
    "RetryAttempts": 0
  }
}


In [9]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_solution_arn
)

user_personalization_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-user-personalization-08866/02d2bad5",
  "ResponseMetadata": {
    "RequestId": "ea94a723-2faa-4114-932f-12348d7419ec",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:10:22 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "127",
      "connection": "keep-alive",
      "x-amzn-requestid": "ea94a723-2faa-4114-932f-12348d7419ec"
    },
    "RetryAttempts": 0
  }
}


###  Popularity Count 솔루션 생성 

In [10]:
# Recipe 선택 
system_popularity_recipe_arn = "arn:aws:personalize:::recipe/aws-popularity-count"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-popularity-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_popularity_recipe_arn,
)



In [11]:
popularity_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-popularity-08866",
  "ResponseMetadata": {
    "RequestId": "70d569cf-27c3-40e7-8a71-270e31009d84",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:10:27 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "101",
      "connection": "keep-alive",
      "x-amzn-requestid": "70d569cf-27c3-40e7-8a71-270e31009d84"
    },
    "RetryAttempts": 0
  }
}


In [12]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = popularity_solution_arn
)

popularity_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-popularity-08866/b9eddfc3",
  "ResponseMetadata": {
    "RequestId": "7b225f9a-516d-4d33-b790-f460191b19f0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:10:32 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "117",
      "connection": "keep-alive",
      "x-amzn-requestid": "7b225f9a-516d-4d33-b790-f460191b19f0"
    },
    "RetryAttempts": 0
  }
}


### Similar ITEMS 솔루션 생성 

In [13]:
 # Recipe 선택 
system_sims_recipe_arn = "arn:aws:personalize:::recipe/aws-similar-items"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-similar-items-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_sims_recipe_arn
)

sims_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-similar-items-08866",
  "ResponseMetadata": {
    "RequestId": "145d2a2a-4f06-46a7-8564-be7c1ba860a2",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:12:28 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "104",
      "connection": "keep-alive",
      "x-amzn-requestid": "145d2a2a-4f06-46a7-8564-be7c1ba860a2"
    },
    "RetryAttempts": 0
  }
}


In [14]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)
sims_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-similar-items-08866/8ef71484",
  "ResponseMetadata": {
    "RequestId": "4044d823-af1e-4f10-afa1-edaa06d0796f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:12:31 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "120",
      "connection": "keep-alive",
      "x-amzn-requestid": "4044d823-af1e-4f10-afa1-edaa06d0796f"
    },
    "RetryAttempts": 0
  }
}


### Personalize Ranking 솔루션 생성 

In [15]:
 # Recipe 선택 
system_ranking_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-ranking-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_ranking_recipe_arn
)

ranking_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-ranking-08866",
  "ResponseMetadata": {
    "RequestId": "6fc47abb-5ada-4925-809d-8b0007f13668",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:12:34 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "98",
      "connection": "keep-alive",
      "x-amzn-requestid": "6fc47abb-5ada-4925-809d-8b0007f13668"
    },
    "RetryAttempts": 0
  }
}


In [16]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = ranking_solution_arn
)
ranking_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-ranking-08866/4fd50926",
  "ResponseMetadata": {
    "RequestId": "160c6bc4-2f5a-4aef-a396-94f785005584",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:12:34 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "114",
      "connection": "keep-alive",
      "x-amzn-requestid": "160c6bc4-2f5a-4aef-a396-94f785005584"
    },
    "RetryAttempts": 0
  }
}


### 모델 생성 확인 하기 
아래 코드 실행하여 모델 생성 여부를 확인합니다. 

In [18]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    ##user_personalization Status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = user_personalization_solution_version_arn
    )  
    status_user_per = describe_solution_version_response["solutionVersion"]["status"]
    print("user_personalization SolutionVersion: {}".format(status_user_per))
    
    ##Popularity Status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = popularity_solution_version_arn
    )  
    status_pop = describe_solution_version_response["solutionVersion"]["status"]
    print("Popularity SolutionVersion: {}".format(status_pop))
    

    #sims status
    describe_solution_version_response = personalize.describe_solution_version(
    solutionVersionArn = sims_solution_version_arn
    )
    status_sims = describe_solution_version_response["solutionVersion"]["status"]
    print("Similar Items SolutionVersion: {}".format(status_sims))
    
    #ranking status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = ranking_solution_version_arn
    )
    status_ranking= describe_solution_version_response["solutionVersion"]["status"]    
    print("Ranking SolutionVersion: {}".format(status_ranking))    
    
    if (status_user_per == "ACTIVE" or status_user_per == "CREATE FAILED") &\
       (status_pop == "ACTIVE" or status_pop == "CREATE FAILED") &\
       (status_sims == "ACTIVE" or status_sims == "CREATE FAILED") &\
       (status_ranking == "ACTIVE" or status_ranking == "CREATE FAILED"):
        break
    print("-------------------------------------->")
    time.sleep(60)

print("All solution creation completed")

user_personalization SolutionVersion: CREATE IN_PROGRESS
Popularity SolutionVersion: CREATE IN_PROGRESS
Similar Items SolutionVersion: CREATE IN_PROGRESS
Ranking SolutionVersion: CREATE IN_PROGRESS
-------------------------------------->
user_personalization SolutionVersion: CREATE IN_PROGRESS
Popularity SolutionVersion: CREATE IN_PROGRESS
Similar Items SolutionVersion: CREATE IN_PROGRESS
Ranking SolutionVersion: CREATE IN_PROGRESS
-------------------------------------->
user_personalization SolutionVersion: CREATE IN_PROGRESS
Popularity SolutionVersion: CREATE IN_PROGRESS
Similar Items SolutionVersion: CREATE IN_PROGRESS
Ranking SolutionVersion: CREATE IN_PROGRESS
-------------------------------------->
user_personalization SolutionVersion: CREATE IN_PROGRESS
Popularity SolutionVersion: CREATE IN_PROGRESS
Similar Items SolutionVersion: CREATE IN_PROGRESS
Ranking SolutionVersion: CREATE IN_PROGRESS
-------------------------------------->
user_personalization SolutionVersion: CREATE IN_

### Store variables

In [19]:

%store user_personalization_solution_arn
%store user_personalization_solution_version_arn

%store popularity_solution_arn
%store popularity_solution_version_arn

%store sims_solution_version_arn
%store sims_solution_arn

%store ranking_solution_version_arn
%store ranking_solution_arn


Stored 'user_personalization_solution_arn' (str)
Stored 'user_personalization_solution_version_arn' (str)
Stored 'popularity_solution_arn' (str)
Stored 'popularity_solution_version_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'sims_solution_arn' (str)
Stored 'ranking_solution_version_arn' (str)
Stored 'ranking_solution_arn' (str)
